In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
prob_hosteler = 0.60
prob_day_scholar = 0.40
prob_A_given_hosteler = 0.30
prob_A_given_day_scholar = 0.20
(prob_hosteler*prob_A_given_hosteler)/((prob_hosteler*prob_A_given_hosteler)+(prob_day_scholar*prob_A_given_day_scholar))

0.6923076923076923

In [4]:
prob_dis=0.01
prob_not_dis=0.99
prob_pos_dis=0.99
prob_pos_no_dis=0.02
(prob_dis*prob_pos_dis)/((prob_dis*prob_pos_dis)+((prob_not_dis*prob_pos_no_dis)))

0.3333333333333333

In [8]:
def fit(X, y):
    classes = y.unique()
    class_probs = y.value_counts(normalize=True).to_dict()
    feature_probs = {}
    for feature in X.columns:
        feature_probs[feature] = {}
        for cls in classes:
            subset = X[y == cls]
            feature_prob = subset.groupby(feature).size() / subset.shape[0]
            feature_probs[feature][cls] = feature_prob.to_dict()
    return class_probs, feature_probs, classes

def predict(X, class_probs, feature_probs, classes):
    predictions = []
    probabilities = []
    for _, row in X.iterrows():
        class_probs_temp = {}
        for cls in classes:
            prob = class_probs[cls]
            for feature in X.columns:
                value = row[feature]
                feature_prob = feature_probs.get(feature, {}).get(cls, {}).get(value, 0)
                prob *= feature_prob
            class_probs_temp[cls] = prob
        total_prob = sum(class_probs_temp.values())
        if total_prob == 0:
            total_prob = 1
        class_probs_normalized = {cls: (prob / total_prob) for cls, prob in class_probs_temp.items()}
        predicted_class = max(class_probs_normalized, key=class_probs_normalized.get)
        predictions.append(predicted_class)
        probabilities.append(class_probs_normalized[predicted_class])
    return predictions, probabilities

data = pd.read_csv('q2.csv')
X = data.drop('buys_computer', axis=1)
y = data['buys_computer']
class_probs, feature_probs, classes = fit(X, y)
test_data = pd.DataFrame([{
    'age': '<=30',
    'income': 'medium',
    'student': 'yes',
    'credit_rating': 'fair'
}])
predictions, probabilities = predict(test_data, class_probs, feature_probs, classes)
print(f'Prediction: {predictions[0]}')
print(f'Probability: {probabilities[0]}')

Prediction: yes
Probability: 0.8350515463917526


In [9]:
def fit(X, y):
    classes = y.unique()
    class_counts = y.value_counts()
    total_count = len(y)
    class_probs = {cls: count / total_count for cls, count in class_counts.items()}
    word_probs = {cls: {} for cls in classes}
    vocabulary = set()
    for cls in classes:
        subset = X[y == cls]
        words = ' '.join(subset).split()
        vocabulary.update(words)
        word_counts = pd.Series(words).value_counts()
        total_words = len(words)
        for word in vocabulary:
            word_probs[cls][word] = (word_counts.get(word, 0) + 1) / (total_words + len(vocabulary))
    return class_probs, word_probs, vocabulary, classes

def predict(X, class_probs, word_probs, vocabulary, classes):
    predictions = []
    confidences = []
    for text in X:
        words = text.split()
        class_probs_score = {}
        for cls in classes:
            prob = class_probs[cls]
            for word in words:
                prob *= word_probs[cls].get(word, 1 / (sum(word_probs[cls].values()) + len(vocabulary)))
            class_probs_score[cls] = prob
        total_score = sum(class_probs_score.values())
        class_probs_normalized = {cls: score / total_score for cls, score in class_probs_score.items()}
        predicted_class = max(class_probs_normalized, key=class_probs_normalized.get)
        predictions.append(predicted_class)
        confidences.append(class_probs_normalized[predicted_class])
    return predictions, confidences

df = pd.read_csv("q3.csv")
class_probs, word_probs, vocabulary, classes = fit(df['text'], df['tag'])
y_true = df['tag']
y_pred, confidences = predict(df['text'], class_probs, word_probs, vocabulary, classes)
test_sentence = ["A very close game"]
predicted_tag, confidence = predict(test_sentence, class_probs, word_probs, vocabulary, classes)
print(f'Prediction: {predicted_tag[0]}')
print(f'Probability: {confidence[0]}')

Prediction: sports
Probability: 0.9417083804295171
